# Static LCA

This notebook explains how to conduct a static (i.e. deterministic; or in other words, ignoring uncertainties) LCA for the pavement structure defined in the case study used in the lectures.

## Content

3.1) Single-LCA: Presentation of the steps needed to calcule the environmental impacts for a product system corresponding to the initial pavement structure.

3.2) Multi-LCA

In [6]:
import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import pandas as pd
from bw2data.query import Filter, Query
from IPython.display import display

In [7]:
#Importing the variables with the project name and background db
from project_details import ei_name, project_name
print(ei_name)
print(project_name)

ecoinvent-391-cutoff
2024_STI_BW25_Lecture


In [8]:
bd.projects.set_current(project_name)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [9]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

In [10]:
fg_name = "asphalt"

In [11]:
# Assign the foreground database to a variable:
fgdb = bd.Database(fg_name)

## 3.1) Single-LCA

## Product system definition

We are going to create a product system for the initial pavement structure. In our foreground database the activity "pavement structure, AC Surf, 0% RAP, regular bitumen" has as reference product the inital pavement structure for which we want to calculate the environmental impacts related to the phases A1-A5. Let's selected it and assign it to the variable "pavement_structure". We can select it by using the .get() method with the activity code (i.e., pavement_structure_1) as argument.

In [19]:
pavement_structure = fgdb.get("pavement_structure_1")

In [20]:
bwa.print_recursive_supply_chain(pavement_structure, max_level=1)

1: 'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None)
  2.01e+06: 'AC Surf, 30% RAP, regular bitumen' (kilogram, NL, None)
  2.72e+06: 'AC Bin' (kilogram, NL, None)
  1.35e+07: 'AC Bin' (kilogram, NL, None)


## Functional unit 

The next step consists of defining the functional unit. This is done by using a dictionary with keys = activities and values = amounts. 
In our case study, the name of the activity is 'pavement_structure' and the amount is 1.

In [21]:
fu = {pavement_structure:1}
fu

{'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None): 1}

## LCIA Methods

n this step we are going to select the LCIA method to be used for calculating the environmental impact scores. In the notebook 2a_Background we explore the different LCIA methods that come with the databse 'biosphere3'. In this example we are going to use the method 'CML v4.8 2016'.

In [22]:
cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
cml_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

## Scores (i.e. results) calculation

In this step we are going to calculate the environmental impact scores. To do so, we need to do the following:

In [23]:
# Create a lca object
lca = bc.LCA(fu)

# Builds matrices, solves the system, generates an LCI matrix.
lca.lci()

results = [] #this list will store the LCA scores
for method in cml_methods:
    lca.switch_method(method)
    lca.lcia() # Characterization, i.e. the multiplication of the elements of the LCI matrix with characterization factors from the chosen LCIA method
    results.append((method[1].title(), 
                    lca.score, # Returns the score, i.e. the sum of the characterized inventory
                    bd.methods.get(method).get('unit')))
    print("Score is {:f} {} for category {}".format(lca.score, 
                                                 bd.methods.get(method).get('unit'),
                                                 method[1].title())
         )
    
results

Score is 4620.867554 kg SO2-Eq for category Acidification
Score is 1451712.044700 kg CO2-Eq for category Climate Change
Score is 305424.681945 kg 1,4-DCB-Eq for category Ecotoxicity: Freshwater
Score is 541236955.229911 kg 1,4-DCB-Eq for category Ecotoxicity: Marine
Score is 4622.939915 kg 1,4-DCB-Eq for category Ecotoxicity: Terrestrial
Score is 35589696.272327 megajoule for category Energy Resources: Non-Renewable
Score is 1131.355488 kg PO4-Eq for category Eutrophication
Score is 101618546.054719 kg 1,4-DCB-Eq for category Human Toxicity
Score is 2.326668 kg Sb-Eq for category Material Resources: Metals/Minerals
Score is 0.053242 kg CFC-11-Eq for category Ozone Depletion
Score is 426.951361 kg ethylene-Eq for category Photochemical Oxidant Formation


[('Acidification', 4620.867554305927, 'kg SO2-Eq'),
 ('Climate Change', 1451712.0447004328, 'kg CO2-Eq'),
 ('Ecotoxicity: Freshwater', 305424.68194525054, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Marine', 541236955.2299111, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Terrestrial', 4622.939915413022, 'kg 1,4-DCB-Eq'),
 ('Energy Resources: Non-Renewable', 35589696.272327244, 'megajoule'),
 ('Eutrophication', 1131.3554877978527, 'kg PO4-Eq'),
 ('Human Toxicity', 101618546.0547189, 'kg 1,4-DCB-Eq'),
 ('Material Resources: Metals/Minerals', 2.3266684801435487, 'kg Sb-Eq'),
 ('Ozone Depletion', 0.053242043984428895, 'kg CFC-11-Eq'),
 ('Photochemical Oxidant Formation', 426.9513612295997, 'kg ethylene-Eq')]

In [24]:
# We can also see the values stores in the list 'results'
results

[('Acidification', 4620.867554305927, 'kg SO2-Eq'),
 ('Climate Change', 1451712.0447004328, 'kg CO2-Eq'),
 ('Ecotoxicity: Freshwater', 305424.68194525054, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Marine', 541236955.2299111, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Terrestrial', 4622.939915413022, 'kg 1,4-DCB-Eq'),
 ('Energy Resources: Non-Renewable', 35589696.272327244, 'megajoule'),
 ('Eutrophication', 1131.3554877978527, 'kg PO4-Eq'),
 ('Human Toxicity', 101618546.0547189, 'kg 1,4-DCB-Eq'),
 ('Material Resources: Metals/Minerals', 2.3266684801435487, 'kg Sb-Eq'),
 ('Ozone Depletion', 0.053242043984428895, 'kg CFC-11-Eq'),
 ('Photochemical Oxidant Formation', 426.9513612295997, 'kg ethylene-Eq')]

In [25]:
#Creates a table with the results
results_df = pd.DataFrame(results, columns=["Impact Category", "Score", "Unit"])
results_df = results_df.set_index("Impact Category")
results_df

,Score,Unit
Impact Category,,
Acidification,4.620868e+03,kg SO2-Eq
Climate Change,1.451712e+06,kg CO2-Eq
Ecotoxicity: Freshwater,3.054247e+05,"kg 1,4-DCB-Eq"
Ecotoxicity: Marine,5.412370e+08,"kg 1,4-DCB-Eq"
Ecotoxicity: Terrestrial,4.622940e+03,"kg 1,4-DCB-Eq"
Energy Resources: Non-Renewable,3.558970e+07,megajoule
Eutrophication,1.131355e+03,kg PO4-Eq
Human Toxicity,1.016185e+08,"kg 1,4-DCB-Eq"
Material Resources: Metals/Minerals,2.326668e+00,kg Sb-Eq


## 3.2) Multi-LCA

In the previous analysis we calculated the environmental impacts of one single product system. But sometimes we may want to compare the environmental impacts of multiple product systems. To illustrate the execution of such comparative analysis let's consider we want to compare the initial pavement structure (pavement_structure_1) with one where the bitumen used in the surface mixture is a polymer modified bitumen. This pavement structure is given by the activity "pavement structure, AC Surf, 30% RAP, modified bitumen" of the foreground database. it has the following code: pavement_structure_2

Let's assign the activities corresponding to the two pavement structures to the variables 'pavement_structure_1' and 'pavement_structure_2', respectively.

In [26]:
pavement_structure_1 = fgdb.get("pavement_structure_1")
pavement_structure_2 = fgdb.get("pavement_structure_2")

Create a list of functional units:

In [27]:
list_fu = [{pavement_structure_1:1}, {pavement_structure_2:1}]

In [29]:
bd.calculation_setups['pavement_structure_1_vs_pavement_structure_2'] = {'inv':list_fu, 'ia':cml_methods}

In [39]:
mlca = bc.MultiLCA("pavement_structure_1_vs_pavement_structure_2")

# `mlca.results`, is a NumPy array of LCA scores,
# with rows of functional units and
# columns of LCIA methods. Ordering is the same as in the `calculation_setup`.
print(mlca.results.shape)
mlca.results

(2, 11)


array([[4.62086755e+03, 1.45171204e+06, 3.05424682e+05, 5.41236955e+08,
        4.62293992e+03, 3.55896963e+07, 1.13135549e+03, 1.01618546e+08,
        2.32666848e+00, 5.32420440e-02, 4.26951361e+02],
       [4.66550304e+03, 1.47012742e+06, 3.12774541e+05, 5.54607998e+08,
        4.69294385e+03, 3.58550783e+07, 1.15959004e+03, 1.01630128e+08,
        2.45825579e+00, 5.29742793e-02, 4.31881990e+02]])

In [37]:
#Creates a table with the results
df = pd.DataFrame(index=cml_methods, columns=["pavement_structure_1", "pavement_structure_2"], data=mlca.results.T)
df

,pavement_structure_1,pavement_structure_2
"(CML v4.8 2016, acidification, acidification (incl. fate, average Europe total, A&B))",4.620868e+03,4.665503e+03
"(CML v4.8 2016, climate change, global warming potential (GWP100))",1.451712e+06,1.470127e+06
"(CML v4.8 2016, ecotoxicity: freshwater, freshwater aquatic ecotoxicity (FAETP inf))",3.054247e+05,3.127745e+05
"(CML v4.8 2016, ecotoxicity: marine, marine aquatic ecotoxicity (MAETP inf))",5.412370e+08,5.546080e+08
"(CML v4.8 2016, ecotoxicity: terrestrial, terrestrial ecotoxicity (TETP inf))",4.622940e+03,4.692944e+03
"(CML v4.8 2016, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels)",3.558970e+07,3.585508e+07
"(CML v4.8 2016, eutrophication, eutrophication (fate not incl.))",1.131355e+03,1.159590e+03
"(CML v4.8 2016, human toxicity, human toxicity (HTP inf))",1.016185e+08,1.016301e+08
"(CML v4.8 2016, material resources: metals/minerals, abiotic depletion potential (ADP): elements (ultimate reserves))",2.326668e+00,2.458256e+00
"(CML v4.8 2016, ozone depletion, ozone layer depletion (ODP steady state))",5.324204e-02,5.297428e-02


Another way to calculate and present the results in a table:

In [41]:
def format_results(mlca):
    formatted_results = []
    for i, scores in enumerate(mlca.results):  # the results for a fu
        for j, method_key in enumerate(mlca.methods):  # the result for each method
            demand = list(mlca.func_units[i].values())[0]
            activity = list(mlca.func_units[i].keys())[0]
            method = bd.Method(method_key)
            a_result = {
                "activity": repr(activity),
                "demand": demand,
                "method": method_key[0],
                "category": method_key[1],
                "subcategory": method_key[2],
                "score": scores[j],
                "unit": method.metadata["unit"],
            }
            formatted_results.append(a_result)
    return formatted_results

In [42]:
df_results = pd.DataFrame(format_results(mlca))
df_results

,activity,demand,method,category,subcategory,score,unit
0,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,acidification,"acidification (incl. fate, average Europe tota...",4.620868e+03,kg SO2-Eq
1,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,climate change,global warming potential (GWP100),1.451712e+06,kg CO2-Eq
2,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,ecotoxicity: freshwater,freshwater aquatic ecotoxicity (FAETP inf),3.054247e+05,"kg 1,4-DCB-Eq"
3,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,ecotoxicity: marine,marine aquatic ecotoxicity (MAETP inf),5.412370e+08,"kg 1,4-DCB-Eq"
4,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,ecotoxicity: terrestrial,terrestrial ecotoxicity (TETP inf),4.622940e+03,"kg 1,4-DCB-Eq"
5,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,energy resources: non-renewable,abiotic depletion potential (ADP): fossil fuels,3.558970e+07,megajoule
6,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,eutrophication,eutrophication (fate not incl.),1.131355e+03,kg PO4-Eq
7,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,human toxicity,human toxicity (HTP inf),1.016185e+08,"kg 1,4-DCB-Eq"
8,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,2.326668e+00,kg Sb-Eq
9,"'pavement structure, AC Surf, 30% RAP, regular...",1,CML v4.8 2016,ozone depletion,ozone layer depletion (ODP steady state),5.324204e-02,kg CFC-11-Eq
